In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df=pd.read_csv("/kaggle/input/dslatam/dataset_SCL.csv") #load dataset
df.head() #show first 5 rows at table

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (1,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,1,1,2017,Domingo,I,American Airlines,Santiago,Miami
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,2,1,2017,Lunes,I,American Airlines,Santiago,Miami
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,3,1,2017,Martes,I,American Airlines,Santiago,Miami
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,4,1,2017,Miercoles,I,American Airlines,Santiago,Miami
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,5,1,2017,Jueves,I,American Airlines,Santiago,Miami


In [2]:
df.shape

(68206, 18)

In [3]:
df.dtypes # show type of variables for every column

Fecha-I      object
Vlo-I        object
Ori-I        object
Des-I        object
Emp-I        object
Fecha-O      object
Vlo-O        object
Ori-O        object
Des-O        object
Emp-O        object
DIA           int64
MES           int64
AÑO           int64
DIANOM       object
TIPOVUELO    object
OPERA        object
SIGLAORI     object
SIGLADES     object
dtype: object

In [4]:
# Adjust variables for correct format
df['Fecha-I']=pd.to_datetime(df['Fecha-I'])
df['Fecha-O']=pd.to_datetime(df['Fecha-O']) 

#Check if object variable can be category
for col in df.columns:
    if df[col].dtype == 'object':
        print(f"{col} has {df[col].nunique()} different values")
        

Vlo-I has 750 different values
Ori-I has 1 different values
Des-I has 64 different values
Emp-I has 30 different values
Vlo-O has 866 different values
Ori-O has 1 different values
Des-O has 63 different values
Emp-O has 32 different values
DIANOM has 7 different values
TIPOVUELO has 2 different values
OPERA has 23 different values
SIGLAORI has 1 different values
SIGLADES has 62 different values


In [5]:
#Based on the data catalog and the information above we have a cut-off for categorical variables with less than 500 values 
#I am doing this because I believe that this is just a sample of a bigger dataset which can contain more destination and origin cities.
for col in df.columns:
    if df[col].dtype == 'object' and df[col].nunique()<200:
        df[col]=df[col].astype('category')

In [6]:
#Considering that this is just a sample and assuming that the original dataset is a CSV file, the right thing to do is to add an argument on the read_csv function, dtypes={dict of every column with the value as the dtype}